<a href="https://colab.research.google.com/github/SahanaRaghu/Machine-Learning-cookbook/blob/master/myWork_MultiClassification_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
from google.colab import files
uploaded = files.upload()

Saving Input_dpci_pog_preprocessed.csv to Input_dpci_pog_preprocessed (5).csv


In [168]:
from google.colab import files
uploadedTest = files.upload()

Saving Input_dpci_pog_test_data.csv to Input_dpci_pog_test_data (6).csv


In [0]:
#Import to dataframe
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
import io
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

input_data = pd.read_csv(io.StringIO(uploaded['Input_dpci_pog_preprocessed.csv'].decode('utf-8')))
test_data = pd.read_csv(io.StringIO(uploadedTest['Input_dpci_pog_test_data.csv'].decode('utf-8')))

input_data = input_data.reindex(
    np.random.permutation(input_data.index))

In [0]:
def preprocess_features(input_data_dataframe, length):
  """Prepares input features from data set.

  Args:
    input_data_dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """

  features = pd.DataFrame()
  features["lengthBy10"] = pd.Series(dtype=float, data=input_data_dataframe["data"].apply(lambda l: len(l))/10)
  features["noOfDigits"] = pd.Series(dtype=float, data=input_data_dataframe["data"].apply(lambda l: getNoOfDigits(l)))
  features["noOfAlphabets"] = pd.Series(dtype=float, data=input_data_dataframe["data"].apply(lambda l: getNoOfAlphas(l)))

  
  #length = input_data_dataframe['data'].apply(lambda l: len(l)).max()
  i = 1
  while(i<=length):
    features[str(i)] = input_data_dataframe['data'].apply(lambda l: 1 if (str.isdigit(l[i-1:i]) and l[i-1:i] != "") else 0 if (l[i-1:i] == "") else 2)
    i = i+1
  
  return features

def getNoOfDigits(val):
    count = 0
    for i in val:
      if i.isdigit() == True: 
        count+= 1
    return count/10
  
def getNoOfAlphas(val):
  count = 0
  for i in val:
    if i.isalpha() == True: 
      count+= 1
  return count/10

def func(x):
    if x == 'dpci':
        return 0
    elif x == 'pog':
        return 1
    elif x == 'backroomlocation':
        return 2
    elif x == 'DCLabels':
        return 3
    elif x == 'aisle':
        return 4
    elif x == 'keyword':
        return 5
    else:
        return 6

def preprocess_targets(input_data_dataframe):
  """Prepares target features (i.e., labels) from data set.

  Args:
    input_data_dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets['data_type'] = input_data_dataframe['target']
  output_targets['data_type'] = output_targets['data_type'].apply(func)

  return output_targets

In [0]:
#---Divide data into training & validation data---

length = input_data['data'].apply(lambda l: len(l)).max()
# Choose the first 28 out of 40 examples for training.
training_examples = preprocess_features(input_data.head(28), length)
training_targets = preprocess_targets(input_data.head(28))

# Choose the last 12 out of 40 examples for validation.
validation_examples = preprocess_features(input_data.tail(12), length)
validation_targets = preprocess_targets(input_data.tail(12))

# Complete data set
all_examples = preprocess_features(input_data, length)
all_targets = preprocess_targets(input_data)

#Test data
test_examples = preprocess_features(test_data, length)
test_targets = preprocess_targets(test_data)

# # Double-check that we've done the right thing.
# print("Training examples summary:")
# display.display(training_examples.head())
# print("Validation examples summary:")
# display.display(validation_examples.head())

# print("Training targets summary:")
# display.display(training_targets.head())

# print("Validation targets summary:")
# display.display(validation_targets.head())

# print("Test data summary:")
# display.display(test_examples.head())
# display.display(test_targets.head())

In [172]:

from sklearn.linear_model import LogisticRegression


## Logistic Regression
logreg = LogisticRegression()
logreg.fit(training_examples, training_targets)
errorTrain = round(logreg.score(training_examples, training_targets) * 100, 2)
print(errorTrain) # error on training data from trained split

errorVL = round(logreg.score(validation_examples, validation_targets) * 100, 2)
print(errorVL) # error on training data from Test split

## Logistic Regression -- Fit again for the complete training set
logregFullTrain = LogisticRegression()
logregFullTrain.fit(all_examples, all_targets)
errorTrainFull = round(logregFullTrain.score(all_examples, all_targets) * 100, 2)
print(errorTrainFull) # error on training data from trained split

print(logregFullTrain)
# LOgistic Regression use the model created to predict for test data
y_test = logregFullTrain.predict(test_examples)

100.0
100.0
100.0
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A c

In [173]:
from joblib import dump, load
dump(logregFullTrain, 'myWork_Classify_Scikit_learn.joblib') 

['myWork_Classify_Scikit_learn.joblib']

In [0]:
model = load('myWork_Classify_Scikit_learn.joblib') 

In [175]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [176]:
# load the model from disk
result = model.score(test_examples, test_targets)
print(result * 100)

100.0


In [177]:
all_examples.head

<bound method NDFrame.head of     lengthBy10  noOfDigits  noOfAlphabets  1  2  3  4  5  6  7 ...  10  11  \
30         0.3         0.2            0.1  2  1  1  0  0  0  0 ...   0   0   
35         0.4         0.2            0.2  2  2  1  1  0  0  0 ...   0   0   
1          0.9         0.9            0.0  1  1  1  1  1  1  1 ...   0   0   
39         0.2         0.1            0.1  2  1  0  0  0  0  0 ...   0   0   
33         0.3         0.2            0.1  2  1  1  0  0  0  0 ...   0   0   
..         ...         ...            ... .. .. .. .. .. .. .. ...  ..  ..   
36         0.3         0.2            0.1  2  1  1  0  0  0  0 ...   0   0   
56         0.6         0.0            0.6  2  2  2  2  2  2  0 ...   0   0   
43         1.9         1.3            0.6  2  2  1  1  1  1  1 ...   1   1   
31         0.3         0.2            0.1  2  1  1  0  0  0  0 ...   0   0   
17         0.7         0.3            0.4  2  1  1  1  2  2  2 ...   0   0   

    12  13  14  15  16  17  18  1